In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel(f'./file/1.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：2


['https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=wheel%2520cylinder',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=Engine%2520Oil%2520Pan']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

times, list_url = 0, []
while len(list_url) < input_.loc[a, 'SKU Count']:
    times += 1
    
    for page in range(input_.loc[a, 'Page']):
        b = 0
        while True:
            b += 1
            print(times, b, page + 1, len(list_url))
            try:
                request_url = f'''{input_.loc[a, 'Url']}&start={page*100}&num=100'''
                
                resp = requests.get(request_url,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10))
                
                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    list_href = html.xpath('//h2[@class="item-headline"]/a/@href')
                    if list_href:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        list_url += [f'https://www.dormanproducts.com/{href.strip()}' for href in list_href]
        list_url = sorted(list(set(list_url)), key=list_url.index)

        # = = = = = = = = = = = = = = =

        if len(list_url) == input_.loc[a, 'SKU Count']:
            break

resp

1 1 1 0
1 2 1 0
1 3 1 0
1 4 1 0
1 5 1 0
1 6 1 0
1 7 1 0
1 8 1 0
1 9 1 0
1 10 1 0
1 11 1 0
1 12 1 0
1 13 1 0
1 14 1 0
1 15 1 0
1 16 1 0
1 17 1 0
1 1 2 100
1 2 2 100
1 3 2 100
1 4 2 100
1 5 2 100
1 6 2 100
1 7 2 100
1 8 2 100
1 9 2 100
1 10 2 100
1 11 2 100
1 12 2 100
1 13 2 100
1 14 2 100
1 15 2 100
1 16 2 100
1 17 2 100
1 18 2 100
1 19 2 100
1 20 2 100
1 21 2 100
1 22 2 100
1 23 2 100
1 1 3 200
1 1 4 299
1 2 4 299
1 3 4 299
1 4 4 299
1 1 5 399
1 2 5 399
1 3 5 399
1 4 5 399
1 5 5 399
1 1 6 498
1 1 7 598
1 1 8 698
1 1 9 797
1 1 10 897
1 2 10 897
1 3 10 897
1 4 10 897
1 5 10 897
1 6 10 897
1 7 10 897
1 8 10 897
1 9 10 897
1 10 10 897
1 11 10 897
1 12 10 897
1 13 10 897
1 14 10 897
1 15 10 897
1 16 10 897
1 1 11 997
1 2 11 997
1 3 11 997
1 4 11 997
1 5 11 997
1 6 11 997
1 7 11 997
1 8 11 997
1 9 11 997
1 1 12 1095
1 2 12 1095
1 1 13 1195
1 2 13 1195
1 3 13 1195
1 4 13 1195
1 5 13 1195
1 6 13 1195
1 7 13 1195
2 1 1 1209
2 2 1 1209
2 3 1 1209
2 4 1 1209
2 5 1 1209
2 6 1 1209
2 7 1 1209
2 1 2

<Response [200]>

In [6]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://www.dormanproducts.com/p-9421-w37425.a...
1,2,https://www.dormanproducts.com/p-21537-w72258....
2,3,https://www.dormanproducts.com/p-50659-w610165...
3,4,https://www.dormanproducts.com/p-47833-w610150...
4,5,https://www.dormanproducts.com/p-50424-w610166...
...,...,...
1209,1210,https://www.dormanproducts.com/p-13351-w610138...
1210,1211,https://www.dormanproducts.com/p-15198-w37314....
1211,1212,https://www.dormanproducts.com/p-17369-w37499....
1212,1213,https://www.dormanproducts.com/p-4797-m134440....


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_url_error.xlsx', index=False)

output_error

""
